In [1]:
import pandas as pd
import logging
import random
import networkx as nx

In [42]:
class GraphConstructor:
    def __init__(
        self,
        followers_paths=["test2.csv","reduce/followerAI.csv","followers_MachineLearning_Final.csv","followers_ChatGPT.csv"],
        data_paths=["reduce/data_Artificial_Intelligence.csv","data_chat_gpt.csv","data_MachineLearning.csv"],
        info_filepath="graph_info.json",
        centralities_filepath="centralities_info.json",
    ):
        
        self.df = pd.concat([
            pd.read_csv(followers_paths[0],dtype={"thread_user_pk": str,"thread_follower_pk": str,}),
            # pd.read_csv(followers_paths[0],dtype={"user_pk": str,"follower_pk": str,"follower_username": str,"follower_count": int,},),
            # pd.read_csv(followers_paths[1],dtype={"user_pk": str,"follower_pk": str,"follower_username": str,"follower_count": int,},),
            # pd.read_csv(followers_paths[2],dtype={"user_pk": str,"follower_pk": str,"follower_username": str,"follower_count": int,},),
        ])

        self.data = pd.concat([
            pd.read_csv(data_paths[0],dtype={"pk": str,"user_pk": str,"caption": str,"like_count": int},),
            # pd.read_csv(data_paths[1],dtype={"pk": str,"user_pk": str,"caption": str,"like_count": int},),
            # pd.read_csv(data_paths[2],dtype={"pk": str,"user_pk": str,"caption": str,"like_count": int},),
        ])
        
        self.graph = nx.DiGraph()
        self.info_filepath = info_filepath
        self.centralities_filepath = centralities_filepath

    def build_graph(self):
        followers_per_user = (
            self.df.groupby("thread_user_pk")["thread_follower_pk"].apply(list).reset_index()
        )
        for _, (user_pk, listf) in followers_per_user.iterrows():
            
            for follower_pk in listf:
                self.graph.add_edge(follower_pk, user_pk)
        for node in self.graph.nodes():
            self.graph.nodes[node]["threshold"] = random.uniform(0, 1)

        return self.graph

    def log_graph_info(self):
        logging.info(f"Numero di nodi del grafo: {self.graph.number_of_nodes()}")
        logging.info(f"Numero di archi del grafo: {self.graph.number_of_edges()}")

In [43]:
gc = GraphConstructor()
gc.build_graph()
graph = gc.graph
print(len(graph.nodes))
print(len(graph.edges))

9645
25187


In [39]:

df  = pd.read_csv(r"test.csv",dtype={"thread_user_pk": str,"thread_follower_pk": str,})

df = df.groupby("thread_user_pk")["thread_follower_pk"].apply(list).reset_index()
df["follower_count"] = df["thread_follower_pk"].str.len()
df = df.sort_values("follower_count", ascending=True)
df

,thread_user_pk,thread_follower_pk,follower_count
1327,64027350185.0,[63088729774.0],1
1529,65167554010.0,[71865395601.0],1
1527,65161385810.0,[63234126535.0],1
1521,65146157119.0,[68188043480.0],1
1520,65145018124.0,[65175018549.0],1
...,...,...,...
219,63086999261.0,"[65020874839.0, 70250491922.0, 63481825052.0, ...",722
777,63400205749.0,"[67293869765.0, 63779537354.0, 66303212436.0, ...",728
1781,66411901180.0,"[63404228454.0, 70369451491.0, 66844103220.0, ...",1414
2033,68109902162.0,"[69438198003.0, 71474884172.0, 63087475282.0, ...",1431


In [40]:
import pandas as pd

def add_new_edges_with_mapping(df: pd.DataFrame, new_edges: list) -> pd.DataFrame:
    mapping = (
        df[['thread_follower_pk']]
        .drop_duplicates(subset='thread_follower_pk')
        .set_index('thread_follower_pk')
        .to_dict('index')
    )
    
    new_rows = []
    for i, (follower, user) in enumerate(new_edges, start=1):
        info = mapping.get(follower, {'follower_count': 0})
        new_rows.append({
            'thread_user_pk': user,
            'thread_follower_pk': follower,
        })
        if i % 1000 == 0:
            print(f"Aggiunto arco: {i}")
    
    new_df = pd.DataFrame(new_rows)
    # Combiniamo il DataFrame originale con quello delle nuove righe
    df_extended = pd.concat([df, new_df], ignore_index=True)
    return df_extended


def add_triadic_closure_edges(df: pd.DataFrame, max_new_edges: int = 100000) -> pd.DataFrame:
    follow_dict = {}
    for _, row in df.iterrows():
        follower = row['thread_follower_pk']
        followed = row['thread_user_pk']
        if follower not in follow_dict:
            follow_dict[follower] = set()
        follow_dict[follower].add(followed)
    
    existing_edges = set(zip(df['thread_follower_pk'], df['thread_user_pk']))
    new_edges = [] 
    reached_limit = False
    print("fine fase 1")
    i = 0
    for A, followed_set in follow_dict.items():
        if reached_limit:
            break
        for B in followed_set:
            if reached_limit:
                break
            if B in follow_dict:
                for C in follow_dict[B]:
                    if len(new_edges) >= max_new_edges:
                        reached_limit = True
                        break
                    if C not in followed_set and (A, C) not in existing_edges:
                        new_edges.append((A, C))
                        existing_edges.add((A, C)) 
                        i+=1
                        print("aggiunto arco", i)

    df_extended = add_new_edges_with_mapping(df_original, new_edges)
    return df_extended

# Esempio di utilizzo:
if __name__ == "__main__":
    followers_path = "test.csv" 
    df_original = pd.read_csv(
        followers_path,
        dtype={
            "user_pk": str,
            "follower_pk": str,
        }
    )
    
    print("Numero archi originali:", len(df_original))
    df_new = add_triadic_closure_edges(df_original, max_new_edges=3000)
    print("Numero archi dopo l'aggiunta:", len(df_new))


Numero archi originali: 22187
fine fase 1
aggiunto arco 1
aggiunto arco 2
aggiunto arco 3
aggiunto arco 4
aggiunto arco 5
aggiunto arco 6
aggiunto arco 7
aggiunto arco 8
aggiunto arco 9
aggiunto arco 10
aggiunto arco 11
aggiunto arco 12
aggiunto arco 13
aggiunto arco 14
aggiunto arco 15
aggiunto arco 16
aggiunto arco 17
aggiunto arco 18
aggiunto arco 19
aggiunto arco 20
aggiunto arco 21
aggiunto arco 22
aggiunto arco 23
aggiunto arco 24
aggiunto arco 25
aggiunto arco 26
aggiunto arco 27
aggiunto arco 28
aggiunto arco 29
aggiunto arco 30
aggiunto arco 31
aggiunto arco 32
aggiunto arco 33
aggiunto arco 34
aggiunto arco 35
aggiunto arco 36
aggiunto arco 37
aggiunto arco 38
aggiunto arco 39
aggiunto arco 40
aggiunto arco 41
aggiunto arco 42
aggiunto arco 43
aggiunto arco 44
aggiunto arco 45
aggiunto arco 46
aggiunto arco 47
aggiunto arco 48
aggiunto arco 49
aggiunto arco 50
aggiunto arco 51
aggiunto arco 52
aggiunto arco 53
aggiunto arco 54
aggiunto arco 55
aggiunto arco 56
aggiunto arco 5

In [41]:
df_new.to_csv("test2.csv")

In [54]:
G = nx.from_pandas_edgelist(df_original, source='follower_pk', target='user_pk', create_using=nx.DiGraph())

# Conta e stampa il numero di nodi e archi
print("Numero di nodi:", G.number_of_nodes())
print("Numero di archi:", G.number_of_edges())

Numero di nodi: 278138
Numero di archi: 308745


In [55]:
G = nx.from_pandas_edgelist(df_new, source='follower_pk', target='user_pk', create_using=nx.DiGraph())

# Conta e stampa il numero di nodi e archi
print("Numero di nodi:", G.number_of_nodes())
print("Numero di archi:", G.number_of_edges())

Numero di nodi: 278138
Numero di archi: 383745
